In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from keras import backend as K

In [8]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [20]:
imgs_path = "Train"
data_list = []
labels_list = []
classes_list = 43
label_size_list = []
for i in range(classes_list):
    i_path = os.path.join(imgs_path, str(i))
    label_size_list.append({i, len(os.listdir(i_path))})
    for img in os.listdir(i_path):
        im = Image.open(i_path +'/'+ img)
        im = im.resize((32,32))
        im = np.array(im)
        data_list.append(im)
        labels_list.append(i)
data = np.array(data_list)
labels = np.array(labels_list)

In [23]:
label_size_list 

[{0, 210},
 {1, 2220},
 {2, 2250},
 {3, 1410},
 {4, 1980},
 {5, 1860},
 {6, 420},
 {7, 1440},
 {8, 1410},
 {9, 1470},
 {10, 2010},
 {11, 1320},
 {12, 2100},
 {13, 2160},
 {14, 780},
 {15, 630},
 {16, 420},
 {17, 1110},
 {18, 1200},
 {19, 210},
 {20, 360},
 {21, 330},
 {22, 390},
 {23, 510},
 {24, 270},
 {25, 1500},
 {26, 600},
 {27, 240},
 {28, 540},
 {29, 270},
 {30, 450},
 {31, 780},
 {32, 240},
 {33, 689},
 {34, 420},
 {35, 1200},
 {36, 390},
 {37, 210},
 {38, 2070},
 {39, 300},
 {40, 360},
 {41, 240},
 {42, 240}]

In [3]:
x_train, x_val, y_train, y_val = train_test_split(data, labels,test_size=0.2, random_state=2)

x_train = x_train/255
x_val = x_val/255

y_train = keras.utils.to_categorical(y_train, 43)
y_val = keras.utils.to_categorical(y_val, 43)

Architecture 1

In [9]:
model = Sequential()
model.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (32, 32, 3)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.5))

model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.5))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(43 , activation = 'softmax')) 

model.compile(loss = 'categorical_crossentropy' , metrics=['accuracy',f1_m], optimizer = "adam")
model.fit(x_train, y_train, epochs=30, verbose=1, validation_data=(x_val, y_val)) 

Epoch 1/30
981/981 [==============================] - 20s 20ms/step - loss: 2.3024 - accuracy: 0.3152 - f1_m: 0.1503 - val_loss: 1.3852 - val_accuracy: 0.5782 - val_f1_m: 0.2004
Epoch 2/30
981/981 [==============================] - 18s 19ms/step - loss: 1.3178 - accuracy: 0.5552 - f1_m: 0.2071 - val_loss: 0.7851 - val_accuracy: 0.7569 - val_f1_m: 0.2358
Epoch 3/30
981/981 [==============================] - 19s 19ms/step - loss: 0.9618 - accuracy: 0.6715 - f1_m: 0.2373 - val_loss: 0.5088 - val_accuracy: 0.8407 - val_f1_m: 0.2606
Epoch 4/30
981/981 [==============================] - 18s 18ms/step - loss: 0.7414 - accuracy: 0.7379 - f1_m: 0.2566 - val_loss: 0.3687 - val_accuracy: 0.8951 - val_f1_m: 0.2867
Epoch 5/30
981/981 [==============================] - 17s 17ms/step - loss: 0.6148 - accuracy: 0.7822 - f1_m: 0.2820 - val_loss: 0.2663 - val_accuracy: 0.9249 - val_f1_m: 0.3286
Epoch 6/30
981/981 [==============================] - 17s 18ms/step - loss: 0.5342 - accuracy: 0.8113 - f1_m: 

Transfer Learning

In [12]:
base_model = VGG16(
    weights='imagenet',
    input_shape=(32, 32, 3),
    include_top=False)

base_model.trainable =  False
inputs = keras.Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(43, activation = 'softmax')(x)
model = keras.Model(inputs,outputs)

adam = Adam(learning_rate=0.001) 
save_model = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=2)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',f1_m])
history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val, y_val), shuffle = True, verbose=1, callbacks=[save_model]) 

Epoch 1/10
245/246 [============================>.] - ETA: 0s - loss: 1.9708 - accuracy: 0.4244 - f1_m: 0.1644
Epoch 1: val_loss improved from inf to 1.47079, saving model to best_model.h5
246/246 [==============================] - 62s 252ms/step - loss: 1.9706 - accuracy: 0.4245 - f1_m: 0.1645 - val_loss: 1.4708 - val_accuracy: 0.5402 - val_f1_m: 0.2057
Epoch 2/10
245/246 [============================>.] - ETA: 0s - loss: 1.3273 - accuracy: 0.5869 - f1_m: 0.2241
Epoch 2: val_loss improved from 1.47079 to 1.19044, saving model to best_model.h5
246/246 [==============================] - 59s 242ms/step - loss: 1.3274 - accuracy: 0.5869 - f1_m: 0.2241 - val_loss: 1.1904 - val_accuracy: 0.6319 - val_f1_m: 0.2427
Epoch 3/10
245/246 [============================>.] - ETA: 0s - loss: 1.1024 - accuracy: 0.6552 - f1_m: 0.2494
Epoch 3: val_loss improved from 1.19044 to 1.01552, saving model to best_model.h5
246/246 [==============================] - 60s 244ms/step - loss: 1.1024 - accuracy: 0.65

Transfer Learning (VGG16) + Data Augmentation

In [13]:
base_model = VGG16(
    weights='imagenet',
    input_shape=(32, 32, 3),
    include_top=False)

base_model.trainable =  False
inputs = keras.Input(shape=(32, 32, 3))

x = base_model(inputs, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(43, activation = 'softmax')(x)
model = keras.Model(inputs,outputs)

adam = Adam(learning_rate=0.001)
datagen = ImageDataGenerator(
        featurewise_center=True,  
        samplewise_center=True,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)
        
datagen.fit(x_train)

save_model = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=2)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',f1_m])
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=30, validation_data=(x_val, y_val), shuffle = True, callbacks=[save_model])

Epoch 1/30
981/980 [==============================] - ETA: 0s - loss: 1.8365 - accuracy: 0.4413 - f1_m: 0.2251
Epoch 1: val_loss improved from inf to 1.56961, saving model to best_model.h5
980/980 [==============================] - 74s 75ms/step - loss: 1.8365 - accuracy: 0.4413 - f1_m: 0.2251 - val_loss: 1.5696 - val_accuracy: 0.4941 - val_f1_m: 0.2524
Epoch 2/30
981/980 [==============================] - ETA: 0s - loss: 1.3086 - accuracy: 0.5775 - f1_m: 0.2843
Epoch 2: val_loss improved from 1.56961 to 1.48819, saving model to best_model.h5
980/980 [==============================] - 77s 78ms/step - loss: 1.3086 - accuracy: 0.5775 - f1_m: 0.2843 - val_loss: 1.4882 - val_accuracy: 0.5168 - val_f1_m: 0.2736
Epoch 3/30
981/980 [==============================] - ETA: 0s - loss: 1.1315 - accuracy: 0.6297 - f1_m: 0.3065
Epoch 3: val_loss improved from 1.48819 to 1.35620, saving model to best_model.h5
980/980 [==============================] - 76s 78ms/step - loss: 1.1315 - accuracy: 0.6297 

Architecture 1 + Data Augmentation

In [14]:
model = Sequential()
model.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (32,32,3)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.5))

model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.5))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(43 , activation = 'sigmoid'))

adam = Adam(learning_rate=0.001)
datagen = ImageDataGenerator(
        featurewise_center=True,  
        samplewise_center=True,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1, 
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)

datagen.fit(x_train)
save_model = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=2)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',f1_m])
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=30, validation_data=(x_val, y_val), shuffle = True, callbacks=[save_model]) 

Epoch 1/30
980/980 [============================>.] - ETA: 0s - loss: 2.6691 - accuracy: 0.2327 - f1_m: 0.1353
Epoch 1: val_loss improved from inf to 3.27958, saving model to best_model.h5
980/980 [==============================] - 20s 20ms/step - loss: 2.6685 - accuracy: 0.2329 - f1_m: 0.1353 - val_loss: 3.2796 - val_accuracy: 0.1598 - val_f1_m: 0.1548
Epoch 2/30
979/980 [============================>.] - ETA: 0s - loss: 1.9292 - accuracy: 0.3839 - f1_m: 0.1796
Epoch 2: val_loss improved from 3.27958 to 2.95583, saving model to best_model.h5
980/980 [==============================] - 19s 20ms/step - loss: 1.9290 - accuracy: 0.3838 - f1_m: 0.1797 - val_loss: 2.9558 - val_accuracy: 0.2045 - val_f1_m: 0.1819
Epoch 3/30
979/980 [============================>.] - ETA: 0s - loss: 1.5988 - accuracy: 0.4724 - f1_m: 0.1987
Epoch 3: val_loss improved from 2.95583 to 2.79477, saving model to best_model.h5
980/980 [==============================] - 19s 20ms/step - loss: 1.5991 - accuracy: 0.4724 

Architecture 2

In [24]:
model = Sequential() 
model.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (32,32,3)))
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization(axis=-1))

model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5)) 
model.add(Dense(43 , activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , metrics=['accuracy',f1_m], optimizer = "adam")
model.fit(x_train, y_train, epochs=15, verbose=1, validation_data=(x_val, y_val)) 

Epoch 1/15
981/981 [==============================] - 80s 81ms/step - loss: 0.3556 - accuracy: 0.9105 - f1_m: 0.9092 - val_loss: 0.0967 - val_accuracy: 0.9711 - val_f1_m: 0.9718
Epoch 2/15
981/981 [==============================] - 79s 80ms/step - loss: 0.0338 - accuracy: 0.9906 - f1_m: 0.9909 - val_loss: 0.0297 - val_accuracy: 0.9913 - val_f1_m: 0.9925
Epoch 3/15
981/981 [==============================] - 83s 85ms/step - loss: 0.0271 - accuracy: 0.9917 - f1_m: 0.9920 - val_loss: 0.0377 - val_accuracy: 0.9895 - val_f1_m: 0.9897
Epoch 4/15
981/981 [==============================] - 78s 79ms/step - loss: 0.0170 - accuracy: 0.9948 - f1_m: 0.9948 - val_loss: 0.0212 - val_accuracy: 0.9935 - val_f1_m: 0.9936
Epoch 5/15
981/981 [==============================] - 78s 79ms/step - loss: 0.0230 - accuracy: 0.9937 - f1_m: 0.9933 - val_loss: 0.0343 - val_accuracy: 0.9909 - val_f1_m: 0.9913
Epoch 6/15
981/981 [==============================] - 82s 84ms/step - loss: 0.0161 - accuracy: 0.9945 - f1_m: 

Architecture 2 + Data Augmentation

In [15]:
model = Sequential()
model.add(Conv2D(16 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (32,32,3)))
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization(axis=-1))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization(axis=-1))

model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5)) 
model.add(Dense(43 , activation = 'sigmoid'))

adam = Adam(learning_rate=0.001)
datagen = ImageDataGenerator(
        featurewise_center=True,  
        samplewise_center=True,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,
        horizontal_flip=True,  
        vertical_flip=False) 

datagen.fit(x_train)
save_model = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=2)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',f1_m])
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=30, validation_data=(x_val, y_val), shuffle = True, callbacks=[save_model])

Epoch 1/30
981/980 [==============================] - ETA: 0s - loss: 1.3138 - accuracy: 0.6324 - f1_m: 0.0962
Epoch 1: val_loss improved from inf to 1.70102, saving model to best_model.h5
980/980 [==============================] - 80s 81ms/step - loss: 1.3138 - accuracy: 0.6324 - f1_m: 0.0962 - val_loss: 1.7010 - val_accuracy: 0.5712 - val_f1_m: 0.0922
Epoch 2/30
981/980 [==============================] - ETA: 0s - loss: 0.2868 - accuracy: 0.9078 - f1_m: 0.1005
Epoch 2: val_loss improved from 1.70102 to 0.45467, saving model to best_model.h5
980/980 [==============================] - 87s 89ms/step - loss: 0.2868 - accuracy: 0.9078 - f1_m: 0.1005 - val_loss: 0.4547 - val_accuracy: 0.8651 - val_f1_m: 0.0944
Epoch 3/30
981/980 [==============================] - ETA: 0s - loss: 0.1900 - accuracy: 0.9371 - f1_m: 0.1007
Epoch 3: val_loss did not improve from 0.45467
980/980 [==============================] - 101s 103ms/step - loss: 0.1900 - accuracy: 0.9371 - f1_m: 0.1007 - val_loss: 0.7944

Conclusions
* Architecture seems to play the biggest role when it comes to this dataset.
* Transfer Learning and Data Augmentation doesn't seem to help with this problem.